# Data Collection - Wildfires & Climate Change 

- developer: name
- personal-use-script: A
- secret: ABC

In [1]:
import praw
import pandas as pd
from datetime import datetime 
import networkx as nx 
import pickle
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
import glob
import matplotlib.pyplot as plt
import matplotlib

In [2]:
reddit = praw.Reddit(client_id='A',
                     client_secret='ABC',
                     user_agent='name',
                     username= 'name', 
                     password= 'name'
                    )

# xxx delete before publishing xxx

In [2]:
reddit = praw.Reddit(client_id='u1l3H359SXM0pQ',
                     client_secret='lq7ouUvkECW6ofAhNS9ai0ZJjJ0',
                     user_agent='lfo_cs_sds20',
                     username= 'lfo_cs_sds20', 
                     password= 'lfo_cs_sds20'
                    )

In [3]:
sub_list = [
 'AskScienceDiscussion',
 'askscience',
 'climatechange',
 'climateskeptics'
]

## Comment Dataframe for multiple Subreddits

* Iterating through a list of subreddits
* Iterating through unlimmited number of hot submissions that contain the keyword "fire" in each subreddit
* Getting the comments for each submission
* Saving the author (if not deleted)
* Saving the comment body (if not deleted)
* Append dataframe-list with comment id, body, author name, upvotes, timestamp, comment level (depth) and parent id and subreddit name

In [ ]:
for sub in sub_list:
    subreddit = reddit.subreddit(sub)
    for c, submission in enumerate(subreddit.hot(limit=None)):
        if "fire" in submission.title:
            for c,comment in enumerate(submission.comments.list()):
                # THe try exists because some reddit comments are from authors who 
                # have deleted their account, but the comments persist. 
                try:
                    x = comment.author.name,
                    authorname = x[0]
                except AttributeError:
                    authorname = "[deleted]"
            
                try:
                    comment_body = comment.body,
                except AttributeError:
                    comment_body = "[deleted]"
            
                try: 
                    df_list.append([ \
                    comment.id,
                    comment_body,
                    authorname,
                    comment.ups,
                    comment.created_utc,
                    comment.depth,
                    comment.parent_id[3:],
                    subreddit
                    ])
                except AttributeError:
                    continue
                    
reddit_df = pd.DataFrame(df_list,columns=["id","body","authorname","ups","created_utc","depth","parent_id","subreddit"])

In [ ]:
# create datetime object
reddit_df["date"] = reddit_df["created_utc"].map(lambda x: datetime.utcfromtimestamp(x))

display(reddit_df.head(10))
reddit_df.shape

## Did this in April 2020 for Australian Wildfires and in October 2020 for Californian Wildfires (With same keyword settings)

In [ ]:
# Store data (serialize)
with open('fire_reddit_df.pickle', 'wb') as handle:
    pickle.dump(reddit_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# also save datafrate as csv 
reddit_df.to_csv('fire_reddit_comments_df.csv', encoding='utf-8', index=False)

## Create comment tree (acyclic graph) for each subreddit


In [ ]:
for sub in sub_list:
    
    G = nx.DiGraph()
    
    node_set = set([])
    
    subreddit = reddit.subreddit(sub)
    
    for submission in subreddit.hot(limit=None):
        if "fire" in submission.title:
            submission.comments.replace_more(limit=None)
            
            for i in submission.comments.list():
                try:
                    G.add_node(i.id, depth=i.depth,name=i.author.name)
                except AttributeError: 
                    G.add_node(i.id, depth=i.depth,name="[Deleted]")
                node_set.add(i.id)
            
            for i in submission.comments.list():
                if i.parent_id and i.parent_id[3:] in node_set:
                    G.add_edge(i.id,i.parent_id[3:])
    
    print(subreddit,G.number_of_nodes(),'nodes')

    nx.write_gexf(G,('%s.gexf'%subreddit))

## Combine Australia and California Data

In [4]:
# Load reddit_df for Australia and California as pickles
with open('aus_fire_reddit_df.pickle', 'rb') as handle:
    reddit_df_aus = pickle.load(handle)

with open('cal_fire_reddit_df.pickle', 'rb') as handle:
    reddit_df_cal = pickle.load(handle)


reddit_df_aus['wave'] = 'australia'  
reddit_df_cal['wave'] = 'california' 
        
reddit_df = reddit_df_aus.append(reddit_df_cal)  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The name modaction has been deprecated. Please import modaction as mod_action. This feature will be removed on the next major version release.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# Store data (serialize)
with open('reddit_df.pickle', 'wb') as handle:
    pickle.dump(reddit_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save datafrate as csv 
reddit_df.to_csv('reddit_df.csv', encoding='utf-8', index=False)

## Find original submission to comment trees

In [7]:
with open('reddit_df.pickle', 'rb') as handle:
    reddit_df = pickle.load(handle)
    
display(reddit_df)    

,id,body,authorname,ups,created_utc,depth,parent_id,subreddit,date,wave
0,fktjz0a,(Yep. You could do this experiment now. Li...,Abyss_of_Dreams,2,1.584523e+09,0,fkmq0d,AskScienceDiscussion,2020-03-18 09:21:44,australia
1,fktka9j,"(Also, this is how those fire blankets work,)",PivotPsycho,1,1.584524e+09,0,fkmq0d,AskScienceDiscussion,2020-03-18 09:28:41,australia
2,fkghpja,(No where near close to the speed of light. It...,thenumber1326,7,1.584160e+09,0,fic5q9,AskScienceDiscussion,2020-03-14 04:28:09,australia
3,fkgqsh5,"(120 meters per second,)",sunceramics,3,1.584170e+09,0,fic5q9,AskScienceDiscussion,2020-03-14 07:12:01,australia
4,fkof6et,(Almost like you could find this answer in a m...,pickles1486,1,1.584386e+09,0,fic5q9,AskScienceDiscussion,2020-03-16 19:13:15,australia
...,...,...,...,...,...,...,...,...,...,...
2175,g2p3p37,"(I said global warming started, not C02 levels...",sennasappel,-1,1.598281e+09,9,g2oztqo,climateskeptics,2020-08-24 14:59:06,california
2176,g2pj4fw,(Just in the last decade the C02 levels have r...,sennasappel,1,1.598289e+09,9,g2phvjl,climateskeptics,2020-08-24 17:03:07,california
2177,g2oyzhi,"(Ah so we're dumb and dishonest. Gotcha.,)",smilodoner,3,1.598279e+09,9,g2oxv3d,climateskeptics,2020-08-24 14:18:49,california
2178,g2lk1ue,"(an opinion piece by the MANN himself, I thoug...",sobakablack,1,1.598200e+09,0,ieptn4,climateskeptics,2020-08-23 16:27:39,california


## Climatechange Australia

In [ ]:
g = nx.read_gexf("climatechange_aus.gexf")
    
threads = pd.Series([g.subgraph(c) for c in nx.weakly_connected_component_subgraphs(g)])
    
subreddit_threads = []
all_trigger_subs_df = []
    
for c,g in enumerate(threads):
    
    comment_branch = list(nx.dag_longest_path(g))
    lpcb_df = reddit_df[reddit_df["id"].isin(comment_branch)]
    temp_sorted = lpcb_df.sort_values("depth",ascending=True)    
    
    count = 0
    for index,row in temp_sorted.iterrows():
        if count < 1:
            c = row.parent_id                         
            subreddit_threads.append(c)
            count += 1
            
    # Run another API call to get trigger submissions    
    
    subs = {}    
        
    for cs in subreddit_threads:
        s = reddit.submission(id = '{}'.format(cs))
        subs[cs] = (s.id, s.title, s.author, s.ups, s.num_comments)    
        cs_df = pd.DataFrame(subs, index=["id","title","author","ups_sub", "n_comments_sub"]).T
        all_trigger_subs_df.append(cs_df)

all_trigger_subs_df = pd.concat(all_trigger_subs_df) 

display(all_trigger_subs_df)

# Store data (serialize)
with open('trigger_climatechange_aus.pickle', 'wb') as handle:
    pickle.dump(all_trigger_subs_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# write to csv
all_trigger_subs_df.to_csv(r'trigger_climatechange_aus.csv', index = True, header=True)

## Climateskeptics Australia

In [17]:
g = nx.read_gexf("climateskeptics_aus.gexf")
    
threads = pd.Series([g.subgraph(c) for c in nx.weakly_connected_component_subgraphs(g)])
    
subreddit_threads = []
all_trigger_subs_df = []
    
for c,g in enumerate(threads):
    
    comment_branch = list(nx.dag_longest_path(g))
    lpcb_df = reddit_df[reddit_df["id"].isin(comment_branch)]
    temp_sorted = lpcb_df.sort_values("depth",ascending=True)    
    
    count = 0
    for index,row in temp_sorted.iterrows():
        if count < 1:
            c = row.parent_id                         
            subreddit_threads.append(c)
            count += 1
            
    # Run another API call to get trigger submissions    
    
    subs = {}    
        
    for cs in subreddit_threads:
        s = reddit.submission(id = '{}'.format(cs))
        subs[cs] = (s.id, s.title, s.author, s.ups, s.num_comments)    
        cs_df = pd.DataFrame(subs, index=["id","title","author","ups_sub", "n_comments_sub"]).T
        all_trigger_subs_df.append(cs_df)

all_trigger_subs_df = pd.concat(all_trigger_subs_df) 

display(all_trigger_subs_df)

# Store data (serialize)
with open('trigger_climateskeptics_aus.pickle', 'wb') as handle:
    pickle.dump(all_trigger_subs_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# write to csv
all_trigger_subs_df.to_csv(r'trigger_climateskeptics_aus.csv', index = True, header=True)

,id,title,author,ups_sub,n_comments_sub
fgbpgq,fgbpgq,CSIRO Bushfire Fact Sheet Makes Claims Not Sup...,Prophetica2020,6,7
fgbpgq,fgbpgq,CSIRO Bushfire Fact Sheet Makes Claims Not Sup...,Prophetica2020,4,7
fgbpgq,fgbpgq,CSIRO Bushfire Fact Sheet Makes Claims Not Sup...,Prophetica2020,4,7
fg0s99,fg0s99,"Coal mine fire in Pennsylvania, burning since ...",None,15,2
fgbpgq,fgbpgq,CSIRO Bushfire Fact Sheet Makes Claims Not Sup...,Prophetica2020,5,7
...,...,...,...,...,...
f4qpxx,f4qpxx,Wildfire Smoke Boosts Photosynthetic Efficiency,sobakablack,22,2
f4s0ow,f4s0ow,What Were the Australian Bushfires Really Abou...,clemaneuverers,15,2
f2pat8,f2pat8,Au: ABC has 'shifted' from 'fire to flood alar...,Kim147,6,2
f16zv5,f16zv5,"As predicted , the rain puts the fires out and...",ox-,54,8


## Climatechange California

In [18]:
g = nx.read_gexf("climatechange_cal.gexf")
    
threads = pd.Series([g.subgraph(c) for c in nx.weakly_connected_component_subgraphs(g)])
    
subreddit_threads = []
all_trigger_subs_df = []
    
for c,g in enumerate(threads):
    
    comment_branch = list(nx.dag_longest_path(g))
    lpcb_df = reddit_df[reddit_df["id"].isin(comment_branch)]
    temp_sorted = lpcb_df.sort_values("depth",ascending=True)    
    
    count = 0
    for index,row in temp_sorted.iterrows():
        if count < 1:
            c = row.parent_id                         
            subreddit_threads.append(c)
            count += 1
            
    # Run another API call to get trigger submissions    
    
    subs = {}    
        
    for cs in subreddit_threads:
        s = reddit.submission(id = '{}'.format(cs))
        subs[cs] = (s.id, s.title, s.author, s.ups, s.num_comments)    
        cs_df = pd.DataFrame(subs, index=["id","title","author","ups_sub", "n_comments_sub"]).T
        all_trigger_subs_df.append(cs_df)

all_trigger_subs_df = pd.concat(all_trigger_subs_df) 

display(all_trigger_subs_df)

# Store data (serialize)
with open('trigger_climatechange_cal.pickle', 'wb') as handle:
    pickle.dump(all_trigger_subs_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# write to csv
all_trigger_subs_df.to_csv(r'trigger_climatechange_cal.csv', index = True, header=True)

,id,title,author,ups_sub,n_comments_sub
jg8jxp,jg8jxp,The current Northern Colorado wildfires are ap...,camawa,1,1
jg8jxp,jg8jxp,The current Northern Colorado wildfires are ap...,camawa,1,1
jg8jxp,jg8jxp,The current Northern Colorado wildfires are ap...,camawa,1,1
jbvwpn,jbvwpn,A tiny pest helped stoke this yearâs devasta...,Independent_Goose551,8,1
jg8jxp,jg8jxp,The current Northern Colorado wildfires are ap...,camawa,1,1
...,...,...,...,...,...
ip27re,ip27re,They Know How to Prevent Megafires. Why Won’t ...,kernals12,83,18
ihii3g,ihii3g,Controlled Burns Can Reduce Wildfires says Sta...,kernals12,71,22
ig9arj,ig9arj,"Yes, climate change is almost certainly fuelin...",_062862,114,14
if3win,if3win,Why the Apparent decline in wildfires up to th...,kernals12,34,5


## Climateskeptics California

In [4]:
g = nx.read_gexf("climateskeptics_cal.gexf")
    
threads = pd.Series([g.subgraph(c) for c in nx.weakly_connected_component_subgraphs(g)])
    
subreddit_threads = []
all_trigger_subs_df = []
    
for c,g in enumerate(threads):
    
    comment_branch = list(nx.dag_longest_path(g))
    lpcb_df = reddit_df[reddit_df["id"].isin(comment_branch)]
    temp_sorted = lpcb_df.sort_values("depth",ascending=True)    
    
    count = 0
    for index,row in temp_sorted.iterrows():
        if count < 1:
            c = row.parent_id                         
            subreddit_threads.append(c)
            count += 1
            
    # Run another API call to get trigger submissions    
    
    subs = {}    
        
    for cs in subreddit_threads:
        s = reddit.submission(id = '{}'.format(cs))
        subs[cs] = (s.id, s.title, s.author, s.ups, s.num_comments)    
        cs_df = pd.DataFrame(subs, index=["id","title","author","ups_sub", "n_comments_sub"]).T
        all_trigger_subs_df.append(cs_df)

all_trigger_subs_df = pd.concat(all_trigger_subs_df) 

display(all_trigger_subs_df)

# Store data (serialize)
with open('trigger_climateskeptics_cal.pickle', 'wb') as handle:
    pickle.dump(all_trigger_subs_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# write to csv
all_trigger_subs_df.to_csv(r'trigger_climateskeptics_cal.csv', index = True, header=True)

,id,title,author,ups_sub,n_comments_sub
jdrl47,jdrl47,US forest fires: NBCNews got drunk and blurted...,pr-mth-s,108,14
jdrl47,jdrl47,US forest fires: NBCNews got drunk and blurted...,pr-mth-s,103,14
jdrl47,jdrl47,US forest fires: NBCNews got drunk and blurted...,pr-mth-s,106,14
jdrl47,jdrl47,US forest fires: NBCNews got drunk and blurted...,pr-mth-s,105,14
jdrl47,jdrl47,US forest fires: NBCNews got drunk and blurted...,pr-mth-s,104,14
...,...,...,...,...,...
inazd9,inazd9,Climate change causes car fires,Kim147,12,2
ige7i8,ige7i8,NASA: Area Burned Globally By Wildfires Droppe...,-mylankovic-,94,8
ig2moa,ig2moa,Climate change is worsening California's helli...,SftwEngr,9,3
ifg7di,ifg7di,The Governor of California believes forest fir...,Long_DuckDonger,215,97
